# Einführung in die Chemieinformatik

---
### Lernziele

- Sie können Smiles mit `rdkit` lesen.
- Sie können Moleküle manipulieren und visualisieren.
- Sie können Deskriptoren für Moleküle berechnen.
- Sie können die Ähnlichkeit von Molekülen anhand von Fingerabdrücken berechnen.

---

Im heutigen Notebook geht es um den Einsatz von Python in der Chemieinformatik. 
Als Fallstudie werden Sie nach einer Alternative zu **Sorafenib** suchen. [Sorafenib](https://de.wikipedia.org/wiki/Sorafenib) ist ein Kinase-Inhibitor, der hauptsächlich bei fortgeschrittenem Nierenkrebs eingesetzt wird.

Wie Sie in der Vorlesung gelernt haben, besteht die erste Hürde in der Chemieinformatik darin, ein geeignetes Format zum Speichern von Molekülstrukturen zu finden. Um Moleküle in Python einzulesen, ist der naheliegendste Weg die Verwendung von *SMILES*. Diese können in Python ohne zusätzliche Bibliotheken gelesen und manipuliert werden. 

Für viele beschriebene Strukturen können Sie die Strukturen in verschiedenen Formaten auf [PubChem](https://pubchem.ncbi.nlm.nih.gov/) erhalten. 
Suchen Sie die SMILES für Sorafenib auf PubChem und kopieren Sie sie in die Zelle unten.


<br>

<details>
<summary><strong>Falls kein Internet vorhanden: HIER klicken</strong></summary>

```python
    sorafenib = "CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC(=C(C=C3)Cl)C(F)(F)F"
```
</details>

In [ ]:
sorafenib = " " # Den Smiles in " " schreiben da sonst dieser nicht richig erkannt wird
print(sorafenib)
type(sorafenib)

Wie Sie sehen können, werden die SMILES als `str` (`string`) gespeichert. Wir können diesen `string` tatsächlich manipulieren und auch Funktionen anwenden.  Das Problem ist jedoch, dass Python zwar die SMILES als `string` versteht, aber die zugrundeliegende molekulare Struktur nicht daraus ableiten kann. Wir haben keine Möglichkeit, Informationen über diese Moleküle zu erhalten.  Mit `len(sorafenib)` kann zwar die Länge des `strings` bestimmt werden, aber es ist beispielsweise nicht möglich, zu ermitteln, aus wie vielen Atomen dieses Molekül besteht.

Hierfür benötigen Sie externe libraries. Eine der meistgenutzten Chemie-Libraries ist RDKit. 
Die wichtigsten Funktionalitäten können mit `from rdkit.Chem import AllChem as Chem` geladen werden. Mit RDKit können Sie Moleküle nicht nur lesen und darstellen, sondern auch manipulieren und Eigenschaften berechnen. 
Um eine SMILES als Molekül zu interpretieren, verwenden Sie `Chem.MolFromSmiles(SMILES)`.

In [ ]:
# Installs RDKit
!pip install rdkit==2022.3.4

In [ ]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import IPythonConsole
sorafenib = Chem.MolFromSmiles(sorafenib)
sorafenib

Mit der Hilfe von RDKit können SMILES eingelesen und als valides Molekül dargestellt werden.
Der `type(sorafenib)` ist jetzt:

In [ ]:
type(sorafenib)

Die Funktion `Chem.MolFromSmiles` wandelt die SMILES-Zeichenfolge in einen neuen Variablentyp um, nämlich den RDKit-Mol. Solange ein Molekül als `rdkit.Chem.rdchem.Mol` in Python gespeichert ist, können Sie alle Funktionen von rdkit auf dieses anwenden. Sie können auch `Chem.MolToSmiles(mol)` verwenden, um das Molekül wieder als SMILES auszugeben:

In [ ]:
Chem.MolToSmiles(sorafenib)

Der SMILES `string` von Sorafenib sieht jetzt anders aus als der, den Sie eingelesen haben. Der Unterschied liegt in der Darstellung der aromatischen Ringe. In der ursprünglichen Zeichenkette wurden explizit Doppelbindungen `=` verwendet, aber stattdessen stehen dort jetzt kleine `c`. RDKit kanonisiert die SMILES automatisch nach einem bestimmten Schema. PubChem verwendet eine andere Strategie für die Kanonisierung. Sie können SMILES in verschiedenen Formen lesen, aber `RDKit` wird immer die gleiche Form ausgeben.

Wenn eine SMILES eingelesen werden soll, der kein gültiges Molekül representiert, wird RDKit einen Fehler ausgeben.

In [ ]:
Chem.MolFromSmiles('CNC(=[O-])c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1') # (=[O-]) anstelle von (=O)

### RDKit
Da Sie nun Sorafenib im richtigen Format haben, können Sie sich auch Informationen zu diesem Molekül ausgeben lassen:

In [ ]:
sorafenib.GetNumAtoms() # Aus wie vielen Atomen besteht Sorafenib

Es gibt verschiedene Funktionen, mit denen man Informationen über Moleküle erhalten kann. `rdkit` weist jedem Atom und jeder Bindung einen Index zu. Mit diesem Index können Sie einzelne Atome oder Bindungen auswählen. Sie können sehen, welches Atom welchen Index hat, indem Sie die `Draw`-Optionen wie folgt ändern:

In [ ]:
from rdkit.Chem import Draw # Draw sublibrary ist für die visuelle Darstellung von Molekülen zuständig
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True 
IPythonConsole.drawOptions.addBondIndices = False
IPythonConsole.molSize = (500, 500) 

In [ ]:
sorafenib

Einzelne Atome können über ihre Indizes mit `.GetAtomWithIdx()` ausgewählt werden. Weitere Funktionen erlauben es, mehr Informationen über die jeweiligen Atome zu erhalten:

In [ ]:
print("Symbol vom Atom mit Index 3")
print(sorafenib.GetAtomWithIdx(3).GetSymbol())

print("\nMasse vom Atom mit dem Index 3")
print(sorafenib.GetAtomWithIdx(3).GetMass())

print("\nHybridisierung vom Atom mit dem Index 3")
print(sorafenib.GetAtomWithIdx(3).GetHybridization())


Mit der Funktion `.SetAtomicNum()` können Sie auch einzelne Atome verändern und z.B. aus dem Keton ein Imin machen.

In [ ]:
sorafenib.GetAtomWithIdx(3).SetAtomicNum(7)
display(sorafenib)
print(Chem.MolToSmiles(sorafenib))
sorafenib.GetAtomWithIdx(3).SetAtomicNum(8)# Veränderung wird wieder Rückgängig gemacht  

Können Sie eines der Fluoratome durch ein Kohlenstoffatom ersetzen?

In [ ]:
sorafenib._____.______ # schreiben Sie hier Ihre Lösung

display(sorafenib)
print(Chem.MolToSmiles(sorafenib))
sorafenib = Chem.MolFromSmiles("CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC(=C(C=C3)Cl)C(F)(F)F")


<details>
<summary><strong>Lösung:</strong></summary>

```python
    sorafenib.GetAtomWithIdx(31).SetAtomicNum(6)
```
</details>

Ähnliche Funktionen können auch für Bindungen verwendet werden. Auch jeder Bindung wird ein Index zugeordnet.

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False # Keine Atom Indizes zeigen
IPythonConsole.drawOptions.addBondIndices = True # Bindungs Indizes zeigen

sorafenib

In [ ]:
print("Welche Art von Bindung ist Bindung 4")
print(sorafenib.GetBondWithIdx(4).GetBondType())

print("\nIst Bindung 4 in einem Ring von Größe 7")
print(sorafenib.GetBondWithIdx(4).IsInRingSize(7))

print("\nIst Bindung 4 in einem Ring von Größe 6")
print(sorafenib.GetBondWithIdx(4).IsInRingSize(6))

IPythonConsole.drawOptions.addBondIndices = False

### Deskriptoren

Hilfreicher als Informationen über einzelne Atome sind Deskriptoren, die für ein ganzes Molekül berechnet werden. Mit verschiedenen Untermodulen in `rdkit` können Sie verschiedene Eigenschaften von Molekülen berechnen: 

In [ ]:
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem.Crippen import MolLogP

print("LogP",MolLogP(sorafenib))
print("Molecular Weight",MolWt(sorafenib))

## Sorafenib Alternativen

Ziel ist es, alternative Moleküle für Sorafenib zu finden. Eine Vorauswahl ist bereits getroffen worden. Die SMILES sind in der Liste `smiles` zu finden.

In [ ]:
smiles = [
    "CNC(=O)c1cc(Oc2ccc(NC(=S)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1",
    "C[C@@H](NC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1)C(=O)NO",
    "CNC(=O)c1cc(Oc2ccc(NC(=S)Nc3cc(C(F)(F)F)cc(C(F)(F)F)c3)cc2)ccn1",
    "N#Cc1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1",
    "CN(C)c1ccc(NC(=O)c2cc(Oc3ccc(NC(=O)Nc4ccc(Cl)c(C(F)(F)F)c4)cc3)ccn2)cc1", 
    "CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Br)c(C(F)(F)F)c3)cc2)ccn1",
    "CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(OC(F)(F)F)cc3)cc2)ccn1",
    "CCNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1",
    "CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3cccc(C(F)(F)F)c3)cc2)ccn1"
]

Um zu vermeiden, dass jedes SMILES einzeln in ein `mol`-Objekt umgewandelt werden muss, schreiben Sie einen `for loop`.

Sie können mehrere Moleküle nebeneinander mit der Funktion `Draw.MolsToGridImage(mols)` anzeigen.

In [ ]:
mols = [ _____ for x in ______] # passen Sie diesen for-loop an
Draw.MolsToGridImage(mols,subImgSize=(300, 300)) # subImgSize ermöglicht es uns, die Moleküle größer zu`drucken`

<details>
<summary><strong>Lösung:</strong></summary>

```python
mols = [Chem.MolFromSmiles(x) for x in smiles]
Draw.MolsToGridImage(mols,subImgSize=(300, 300))
```
</details>

Um unnötige Kosten zu vermeiden, sollten Sie nur die vielversprechendsten Moleküle auswählen. Hierfür können Sie das bisher Gelernt anwenden. 
Eine einfache, aber wichtige Faustregel für die Entwicklung von Arzneimitteln ist die ["Lipinski's Rule of Five"](https://flexikon.doccheck.com/de/Lipinski%27s_rule_of_five). Sie besagt, dass Moleküle die vier bestimmte  Eigenschaften haben, sich besonders gut als Wirkstoff eignen:

#### Lipinski's Rule of Five

>- Nicht mehr als 5 Wasserstoffbrückenbindungs-Donatoren (hauptsächlich Stickstoff-Wasserstoff- und Sauerstoff-Wasserstoff-Bindungen).
>- Nicht mehr als 10 Wasserstoffbrückenbindungs-Akzeptoren (hauptsächlich Sauerstoff- und Stickstoffatome).
>- Ein Molekulargewicht von weniger als 500 g/mol.
>- Ein [logP](https://de.wikipedia.org/wiki/Octanol-Wasser-Verteilungskoeffizient) Wert von weniger als 5.

Sie haben bereits den LogP Wert und das Molekulargewicht mit `rdkit`-Funktionen berechnet.
Das Submodul `Lipinski` in RDKit bietet noch mehr Funktionen u.a. zur Berechnung der Anzahl der Wasserstoffbrückendonatoren und -akzeptoren. 

Berechnen Sie zunächst die Anzahl der Wasserstoffdonatoren (`NumHDonors`)

In [ ]:
from rdkit.Chem.Lipinski import NumHAcceptors, NumHDonors

NumDonors = [______(x) for x in ______] # passen Sie diesen for-loop an

<details>
<summary><strong>Lösung:</strong></summary>

```python
NumDonors = [NumHDonors(x) for x in mols]
```
</details>

Um die Anzahl der Donatoren zusammen mit den Molekülen anzuzeigen, können Sie die Funktion `MolsToGridImage()` verwenden. Hier muss man einfach die `NumDonors` an die Inputvariable `legends` übergeben. Das Problem ist, dass die Funktion immer `Strings` erwartet. Also keine `integer`. Deshalb benutzen wir eine weitere `for-loop`, um den `int` in `NumDonors` in einen `str` umzuwandeln.

In [ ]:
NumDonors = [str(x) for x in NumDonors] # Sie konvertieren die Anzahl der Donoren zum str damit wir sie als Legende benutzen können
Draw.MolsToGridImage(mols, legends = NumDonors,subImgSize = (300,300))

In der Bildunterschrift sehen Sie die Anzahl der Wasserstoffbrückenbindungs-Donatoren. Alle Moleküle haben weniger Donatoren als die von Lipinski angegebene Obergrenze. Wir können das Gleiche für die Akzeptoren wiederholen.
Diesmal schreiben Sie jedoch den `for-loop` so, dass die `integer` sofort in `strings` umgewandelt werden. Auf diese Weise spart man sich einen `for-loop`.

In [ ]:
NumAcceptors = [str(________) for x in ________] # passen Sie diesen for-loop an
Draw.MolsToGridImage(mols, legends = NumAcceptors, subImgSize= (300,300))

<details>
<summary><strong>Lösung:</strong></summary>

```python
NumAcceptors = [str(NumHAcceptors(x)) for x in mols]
Draw.MolsToGridImage(mols, legends = NumAcceptors, subImgSize= (300,300))
```
</details>

Auch hier verstößt keines der Moleküle gegen Lipinskis Regel.
Berechnen Sie nun das Molekulargewicht (`MolWt()`) für die Alternativen zu Sorafenib.

In [ ]:
molWeight = [str(_________) for __ in _______] # passen Sie diesen for-loop an
Draw.MolsToGridImage(mols, legends = molWeight, subImgSize=(300, 300))

<details>
<summary><strong>Lösung:</strong></summary>

```python
molWeight = [str(MolWt(x)) for x in mols]
Draw.MolsToGridImage(mols, legends = molWeight, subImgSize=(300, 300))
```
</details>

Einige Moleküle sind tatsächlich schwerer als "Lipinskis Rule of Five" erlaubt.

Als Letztes berechnen Sie noch den LogP (`MolLogP()`).

In [ ]:
logP = [_______ for ____ in _____] # passen Sie diesen for-loop an. Achten Sie auch auf die str() Funktion
Draw.MolsToGridImage(mols, legends = logP,subImgSize=(300, 300))

<details>
<summary><strong>Lösung:</strong></summary>

```python
logP = [str(MolLogP(x)) for x in mols]
Draw.MolsToGridImage(mols, legends = logP,subImgSize=(300, 300))
```
</details>

Tatsächlich überschreiten die meisten Moleküle Lipinskis LogP-Wert. Nur drei Moleküle haben einen Wert von weniger als fünf. Diese letzten beiden Moleküle sind die einzigen, die alle vier Lipinski-Regeln erfüllen. Daher könnten sie besonders gut als Arzneimittel eignen. Aber man sollte nicht alle anderen Moleküle nur wegen eines zu hohen LogP-Wertes verwerfen. 

Sorafenib selbst hat auch einen LogP-Wert von mehr als 5. Daher entfernen wir nur Moleküle, die sowohl einen zu hohen LogP-Wert als auch ein zu hohes Gewicht haben. Sie können `booleans` verwenden, um die richtigen Moleküle auszuwählen, z. B. `LogP < 5`. Zuvor haben wir jedoch die Werte als `str` gespeichert, um sie unter den Molekülen als Legende auszugeben. Daher berechnen wir die Werte erneut, ohne sie in eine `str` umzuwandeln. Außerdem müssen Sie auch die Listen in ein `array` umwandeln, sonst können Sie die Werte nicht vergleichen.

In [ ]:
import numpy as np

logP = [MolLogP(x) for x in mols]
logP = np.array(logP) # die Liste muss erst noch in einen Array konvertiert werden
logP < 5.0

Machen Sie nun dasselbe für das Gewicht (`MolWt()`).

In [ ]:
molWeight = [____() ___ ____ ___ ___] # passen Sie diesen for-loop an
molWeight = ______________ # konvertieren Sie die Liste in ein Array
molWeight < 500

<details>
<summary><strong>Lösung:</strong></summary>

```python
molWeight = [MolWt(x) for x in mols]
molWeight = np.array(molWeight)
molWeight < 500
```
</details>

Um die Moleküle auszuwählen, die entweder ein Gewicht unter 500 oder einen logP unter fünf haben, können Sie das Symbol `|` verwenden. Das `|` steht für "oder". Die Anweisung `(logP < 5) | (molWeight<500)` wird `True` für Elemente ausgeben, die mindestens eine der beiden Bedingungen erfüllen. `False` wird ausgegeben, wenn ein Element keine der beiden Bedingungen erfüllt. Es bedeutet also, dass entweder ein logP kleiner als fünf oder ein Gewicht kleiner als 500 g/mol ist.

In [ ]:
(logP < 5) | (molWeight<500)

Mit diesem `bool` array können wir jetzt die Moleküle auswählen die noch übrig bleiben.

In [ ]:
mols=np.array(mols)
mols_subset=mols[(logP < 5) | (molWeight<500)] # wir konvertieren die `mol` Liste auch zu einem array
Draw.MolsToGridImage(mols_subset, subImgSize=(300, 300))

Durch die Berechnung von Deskriptoren können Sie die Anzahl der in Frage kommenden Moleküle reduzieren. Im nächsten Schritt werden Sie lernen, wie Sie die Auswahl mit einer Ähnlichkeitssuche weiter reduzieren können.

## Fingerabdrücke & Ähnlichkeitssuche

RDKit kann auch verschiedene *molekulare Fingerabdrücke* berechnen. Dazu gehört der *Extended Connectivity Fingerprint* (ECFP), ursprünglich entwickelt von [Hahn et al.](https://pubs.acs.org/doi/10.1021/ci100050t) 2010. 
RDKit verfügt über eine modifizierte Version, die sie *Morgan Fingerprint* nennen.

Sie können die Funktion `Chem.GetMorganFingerprint(mol,radius)` verwenden, um den ECFP zu berechnen. Um die Ähnlichkeit von Fingerabdrücken zu bewerten, bietet RDKit ebenfalls eine Funktion. Mit `DataStructs.TanimotoSimilarity(fp1,fp2)` kann zum Beispiel die Tanimoto Similarity berechnet werden.

Sehr ähnliche (aber nicht unbedingt identische) Moleküle haben eine Tanimoto-Ähnlichkeit von `1.0` und sehr unähnliche Moleküle haben eine Tanimoto-Ähnlichkeit von `0.0`.

In [ ]:
from rdkit import DataStructs
fp_sorafenib = Chem.GetMorganFingerprint(sorafenib,radius=2)
fp_sorafenib

Der Morganfingerprint wird nicht als reguläres `np.array` gespeichert. Aber in den nächsten Wochen werden Sie lernen, wie Sie auch die *normalen* Vektoren der Fingerabdrücke erhalten können.

Sie haben den Fingerabdruck für Sorafenib berechnet, aber um die Ähnlichkeit zu berechnen, brauchen Sie auch die Fingerabdrücke der anderen Moleküle.

*Schreibe eine `for-loop`, die den Fingerabdruck für alle Moleküle in `mols_subset` berechnet und sie als Liste in `fp_mols` speichert.*

In [ ]:
fp_mols = [Chem.GetMorganFingerprint( ___ ,radius = 2) for __ in ___ ]

<details>
<summary><strong>Lösung. HIER klicken</strong></summary>

```python
    fp_mols = [Chem.GetMorganFingerprint( x ,radius = 2) for x in mols_subset]
```
</details>
<br>
Um die Similarity zu berechnen, verwenden Sie die oben beschriebene Funktion `TanimotoSimilarity(fp1, fp2)`





In [ ]:
DataStructs.TanimotoSimilarity(fp_sorafenib,fp_mols[5])

Schreiben Sie einen `for-loop` der für jedes Moleküle in `fp_mols` die Similarity zu Sorafenib berechnet.

In [ ]:
sorafenib_similarity = [DataStructs.TanimotoSimilarity(___ , ___ ) for x in ____]
sorafenib_similarity

<details>
<summary><strong>Lösung. HIER klicken</strong></summary>

```python
    sorafenib_similarity=[DataStructs.TanimotoSimilarity(fp_sorafenib, x) for x in fp_mols]

```
</details>
<br>


In [ ]:
Draw.MolsToGridImage(mols_subset,legends = [str(x) for x in sorafenib_similarity],subImgSize=(300, 300))

Oben sehen Sie die Ähnlichkeit der einzelnen Moleküle mit Sorafenib. Eine häufig verwendete Faustregel besagt, dass die Moleküle ab einer Ähnlichkeit von 0,8 ähnlich genug sind, um als relevante Alternative betrachtet zu werden. In unserem Fall bedeutet dies, dass wir nur ein Molekül testen würden. Das Molekül mit einer Ähnlichkeit von `0,7959...` würde wahrscheinlich immer noch als relevant angesehen werden.
Grundsätzlich wird eine Ähnlichkeitssuche oft verwendet, um den Raum der relevanten Moleküle einzugrenzen.

Ein medizinischer Chemiker könnte wahrscheinlich auch ohne Computer aus neun Molekülen die relevantesten heraussuchen. Der wirkliche Vorteil des Computers wird erst deutlich, wenn man einen Pool von mehreren Millionen Molekülen hat. Der Code würde genau so funktionieren, man müsste nur die zusätzlichen SMILES am Anfang des Notizbuchs einlesen.


## Übungsaufgabe: Alternativen zum Norfloxacin Antibiotikum

**Zur Benotung einreichen!**

Als etwas anspruchsvollere Aufgabe sollen Sie nun das bisher Gelernte noch einmal selbstständig anwenden. 
Im Grunde ist die Aufgabe den vorherigen sehr ähnlich, aber Sie werden weniger Hilfe erhalten.
Suchen Sie zunächst den SMILES-`string` für Norfloxacin bei [PubChem](https://pubchem.ncbi.nlm.nih.gov/). Konvertieren Sie dann die Zeichenkette in das `mol`-Format und lassen Sie sich das Moleküle anzeigen.

> "Norfloxacin ist ein synthetisches Breitband-Antibiotikum aus der Gruppe der Gyrasehemmer zur oralen Einnahme." 

Source: [Wikipedia](https://de.wikipedia.org/wiki/Norfloxacin)


In [ ]:
# Wenn Sie diese Aufgabe zu einem späteren Zeitpunkt erledigen, können Sie diese Zelle verwenden, 
# um die erforderlichen Bibliotheken auf einmal zu importieren
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Descriptors import MolWt 
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.Lipinski import NumHAcceptors, NumHDonors
from rdkit import DataStructs

In [ ]:
norfloxacin = "CCN1C=C(C(=O)C2=CC(=C(C=C21)N3CCNCC3)F)C(=O)O"
# konvertieren Sie den string zum mol Format
norfloxacin = 
# lassen Sie sich das Molekül anzeigen
norfloxacin

Berechnen Sie anschließend die Deskriptoren für Norfloxacin, die für die *Lipinski's Rule of Five* wichtig sind.

In [ ]:
# Berechnen Sie MW
MW_norfloxacin = 

# Berechnen Sie die Anzahl an H-Brücken Akzeptoren
NumHAcceptors_norfloxacin = 

# Berechnen Sie die Anzahl an H-Brücken Donoren
NumHDonors_norfloxacin = 

# Berechen Sie logP
logP_norfloxacin = 

Die nächste Zelle gibt Ihnen die berechneten Deskriptoren aus:

In [ ]:
print("MW:", MW_norfloxacin)
print("NumHAcceptors", NumHAcceptors_norfloxacin)
print("NumHDonors", NumHDonors_norfloxacin)
print("LogP",logP_norfloxacin)

Eine Vorauswahl möglicher Norfloxacin-Alternativen ist in der nächsten Zelle aufgeführt. Konvertieren Sie die SMILES in das `mol`-Format und berechnen Sie anschließend die Deskriptoren. Am einfachsten geht das mit der oben mehrfach verwendeten Notation `List = [Function(x) for x in OtherList]`.

In [ ]:
# Vergessen Sie nicht, zuerst diese Zelle auszuführen.
quinolones = ["C1CC1N2C=C(C(=O)C3=CC(=C(C=C32)N4CCNCC4)F)C(=O)O",
             "CN1CCN(CC1)C2=C(C=C3C(=C2F)N(C=C(C3=O)C(=O)O)CCF)F",
             "CCN1C=C(C(=O)C2=CC(=C(C(=C21)F)N3CCNC(C3)C)F)C(=O)O",
             "CC1CCC2=C3N1C=C(C(=O)C3=CC(=C2N4CCC(CC4)O)F)C(=O)O",
             "CC1COC2=C3N1C=C(C(=O)C3=CC(=C2N4CCN(CC4)C)F)C(=O)O"
             "CCN1C=C(C(=O)C2=CC(=C(C=C21)N3CCN(CC3)C)F)C(=O)O",
             "CN1CCN(CC1)C2=C(C=C3C4=C2SCCN4C=C(C3=O)C(=O)O)F",
             "CCN1C=C(C(=O)C2=CC(=C(N=C21)N3CCNCC3)F)C(=O)O",
             "CNC1CCCN(C1)C2=C(C=C3C(=C2OC)N(C=C(C3=O)C(=O)O)C4CC4)F",
             "CC1CN(CCN1)C2=C(C(=C3C(=C2)N(C=C(C3=O)C(=O)O)C4CC4)C)F",
             "C[C@H]1COC2=C3N1C=C(C(=O)C3=CC(=C2N4CCN(CC4)C)F)C(=O)O",
             "C[C@H]1COC2=C3N1C=C(C(=O)C3=CC(=C2C4(CC4)N)F)C(=O)O",
             "C[C@@H]1CN(C[C@@H](N1)C)C2=C(C(=C3C(=C2F)N(C=C(C3=O)C(=O)O)C4CC4)N)F",
             "CC1CN(CCN1)C2=C(C=C3C(=C2)N(C=C(C3=O)C(=O)O)C4=C(C=C(C=C4)F)F)F",
             "C1CN(CC1N)C2=C(C=C3C(=O)C(=CN(C3=N2)C4=C(C=C(C=C4)F)F)C(=O)O)F"]

In [ ]:
# Formen Sie die Strings zum mol Format um 
quinolones = 

In [ ]:
# Berechnen Sie hier die vier Deskriptoren für alle Moleküle der Liste
MW_quinolones = 
NumHAcceptors_quinolones = 
NumHDonors_quinolones = 
logP_quinolones = 

In der folgenden Zelle können Sie die Moleküle mit den berechneten Deskriptoren anzeigen. Sie müssen dabei nicht unbedingt den Code nachvollziehen können. 

Sie müssen die Werte selbst ablesen. (`Strg` + Mausrad zum vergrößern.)

In [ ]:
legend = []
for i in range(len(MW_quinolones)):
    legend.append("MW: "+str(round(MW_quinolones[i]))+"\n"+
                 "NumHAcceptors: "+str(NumHAcceptors_quinolones[i])+"\n"+
                 "NumHDonors: "+str(NumHDonors_quinolones[i])+"\n"+
                 "logP: "+str(round(logP_quinolones[i], 4)))

Draw.MolsToGridImage(quinolones, molsPerRow=3, legends = legend,
                    subImgSize=(250,150), useSVG=True)

Da praktisch alle Moleküle der *Lipinski's Rule of Five* folgen, verzichten wir an dieser Stelle darauf, Moleküle aus unserer Datenbank zu entfernen und berechnen stattdessen direkt die Ähnlichkeit zu Norfloxacin. Dazu müssen zunächst die Fingerabdrücke und dann die Tanimoto-Ähnlichkeit berechnet werden.

In [ ]:
# Berechnen Sie zuerst die Fingerprints der Quinolone und von Norfloxacin
norfloxacin_fp = 
quinolones_fp = 

Berechnen Sie nun die Ähnlichkeiten der `quinolones` mit Norfloxacin. 

In der Zelle unten können Sie die Ähnlichkeiten anzeigen.

In [ ]:
quinolones_similarity = 

In [ ]:
Draw.MolsToGridImage(quinolones, legends = [str(round(x, 2)) for x in quinolones_similarity],
                    subImgSize=(250,200), useSVG=True)

Wie Sie sehen können, sind die meisten Moleküle Norfloxacin nicht besonders ähnlich (zumindest laut der Tanimoto Similarity). Tatsächlich handelt es sich jedoch bei jedem der Moleküle um Breitbandantibiotika, die zumindest in der Vergangenheit verfügbar waren. Daher kann man sich nicht allein auf die Ähnlichkeit zwischen den Molekülen verlassen, um auf die Aktivität zu schließen.